In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset, Split
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import re
from collections import Counter
import os
import sys

current_path = os.path.abspath('.')
project_name = 'TinyStoriesProject'
project_path = os.path.join(current_path.split(project_name)[0], project_name)
print(project_path)

C:\Users\peter\Documents\SJSU\DeepLearning\TinyStoriesProject


# Load Dataset

In [2]:
#Load Small Sample for testing
df = pd.read_json("./TinyStories_all_data/data00.json")

In [3]:
train_dataset = load_dataset("./TinyStories_all_data")

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [4]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['story', 'instruction', 'summary', 'source'],
        num_rows: 4967871
    })
})

## Tokenizer

In [5]:
texts = df['story'].dropna().tolist()
# Keep only non-empty, non-whitespace-only strings
texts = [text for text in texts if len(text.strip().split()) > 0]

In [10]:
from tokenizers import BertWordPieceTokenizer

# Initialize
custom_tokenizer = BertWordPieceTokenizer()

# Train
custom_tokenizer.train_from_iterator(
    texts,
    vocab_size=8000,  # You can go lower for a small model
    min_frequency=2,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

In [11]:
from transformers import PreTrainedTokenizerFast

# Wrap Tokenizer
custom_tokenizerfast = PreTrainedTokenizerFast(tokenizer_object=custom_tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>"
)

In [9]:
#Save trained tokenizer
custom_tokenizer.save_model("./custom_tokenizer/")

['./custom_tokenizer/vocab.txt']

In [ ]:
# Load saved toeknaizer
custom_tokenizerfast = PreTrainedTokenizerFast(
    vocab_files_name="custom_tokenizer/vocab.txt",  # generated by save_model
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<s>",
    sep_token="</s>",
    mask_token="<mask>"
)

In [12]:
import random

def mask_random_word(sentence):
    words = sentence.split()
    random_idx = random.randint(0, len(words) - 1)
    
    words[random_idx] = '[MASK]'
    return ' '.join(words)

masked_sentences = [mask_random_word(sentence) for sentence in texts]

In [13]:
from transformers import BertForMaskedLM, Trainer, TrainingArguments

# Load BERT model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
custom_tokenizerfast.pad_token = '[PAD]'

In [15]:
# Tokenize your masked dataset
encodings = custom_tokenizerfast(masked_sentences, padding=True, max_length=512)

C:\Users\peter\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:2718: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [19]:
type(encodings)

transformers.tokenization_utils_base.BatchEncoding

In [20]:
from datasets import Dataset

# Create a Hugging Face Dataset from a list of dicts
train_dataset = Dataset.from_dict(encodings)

MemoryError: 

In [16]:
# Set up the trainer
training_args = TrainingArguments(
    output_dir='./results',  # output directory
    num_train_epochs=3,      # number of training epochs
    per_device_train_batch_size=8,  # batch size
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encodings,
)

In [17]:
trainer.train()

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,head_mask,inputs_embeds,encoder_hidden_states,encoder_attention_mask,labels,output_attentions,output_hidden_states,return_dict,label_ids,label,labels.